In [1]:
import skfem
import skfem.helpers

In [2]:
mesh = skfem.Mesh.load("cube.msh")

Failure to parse tags from meshio.


In [3]:
mesh

<skfem MeshTet1 object>
  Number of elements: 289381
  Number of vertices: 48526
  Number of nodes: 48526
  Named subdomains [# elements]: air [217700], material [71681]

### Solving PDE in weak form

$$
\int_{\Omega} \nabla u \cdot \nabla v \ \mathrm{d}x = \int_{\Omega} \mathbf{M} \cdot \nabla v \ \mathrm{d}x
$$

If we consider same piece-wise-linear function space for both magnetic scalar potential $u$ and components of $\mathbf{M}$, we can re-write the PDE as:
$$
\sum_{j=1}^{n} \sum_{i=1}^{n} v_j u_i \int_{\Omega_{ji}} \nabla\phi_{j} \cdot \nabla\phi_{i} \ \mathrm{d}x = \sum_{j=1}^{n} \sum_{k=1}^{n} v_j m_k \int_{\Omega_{jk}} \nabla\phi_{j} \cdot \hat{\phi_{k}} \ \mathrm{d}x
$$
This can be further re-written as a matrix-vector product:
$$
\mathbf{v}_{j}^{\mathrm{T}} \cdot \mathbf{{S}}_{ji} \cdot \mathbf{u}_{i} = \mathbf{v}_{j}^{\mathrm{T}} \cdot \mathbf{{B}}_{jk} \cdot \mathbf{m}_{k}
$$
Here $\mathbf{u}_{i}$ is the unknown vector corresponding to the values of scalar potential at nodes of the mesh. Thus, we obtain a system of linear equations:
$$
\mathbf{{S}}_{ji} \cdot \mathbf{u}_{i} = \mathbf{{B}}_{jk} \cdot \mathbf{m}_{k}
$$
Where,
$$
\begin{aligned}
    \mathbf{{S}}_{ji} &= \int_{\Omega_{ji}} \nabla\phi_{j} \cdot \nabla\phi_{i} \ \mathrm{d}x \\
    \mathbf{{B}}_{jk} &= \int_{\Omega_{jk}} \nabla\phi_{j} \cdot \hat{\phi_{k}} \ \mathrm{d}x
\end{aligned}
$$
are both system matrices which remain constant for a given mesh and can be assembled first using `scikit-fem` library.

In [4]:
function_space_scalar = skfem.Basis(mesh, skfem.ElementTetP1())

In [5]:
function_space_vector = skfem.Basis(mesh, skfem.ElementVectorH1(skfem.ElementTetP1()))

Define bilinear form for the assembly of stiffness matrix $\mathbf{S}_{ji}$

In [6]:
@skfem.BilinearForm
def stiffness_matrix(u, v, _):
    return skfem.helpers.dot(skfem.helpers.grad(u), skfem.helpers.grad(v))

Define bilinear form for the assembly of the mixed-form matrix $\mathbf{B}_{jk}$

In [7]:
@skfem.BilinearForm
def mixed_form_matrix(u, v, _):
    return skfem.helpers.dot(u, skfem.helpers.grad(v))

In [8]:
%%timeit
skfem.asm(stiffness_matrix, function_space_scalar)

338 ms ± 7.58 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


`skfem.asm` actually calculates the integrals using quadrature points in the mesh and assembles the sparse matrices. 

In [9]:
S = skfem.asm(stiffness_matrix, function_space_scalar)

In [10]:
%%timeit
skfem.asm(mixed_form_matrix, function_space_vector, function_space_scalar)

971 ms ± 24.6 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [11]:
B = skfem.asm(mixed_form_matrix, function_space_vector, function_space_scalar)

In [12]:
S

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 733188 stored elements and shape (48526, 48526)>

In [13]:
B

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 2199189 stored elements and shape (48526, 145578)>

## Create initial magnetisation



In [14]:
import numpy as np

In [15]:
def mag_sat(x):
    m_x = np.zeros_like(x[0])
    m_y = np.zeros_like(x[1])
    m_z = np.zeros_like(x[2])
    m_z[(np.abs(x[0]) < 25.) & (np.abs(x[1]) < 25.) & (np.abs(x[2]) < 25.)] = 1.76 # T

    return np.stack(np.array([m_x, m_y, m_z]), axis=-1)

In [16]:
mag_init_arr = mag_sat(mesh.p)

In [17]:
mag_init_arr.shape

(48526, 3)

Unfortunately, `scikit-fem` works with flattened arrays of vector fields :(

In [19]:
mag_init_flat = np.hstack(mag_init_arr)

In [20]:
mag_init_flat.shape

(145578,)

In [23]:
%%timeit
B @ mag_init_flat

1.54 ms ± 12 μs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


> **_NOTE:_**  As expected, multiplication of a sparse matrix $\mathbf{B}_{jk}$ with $\mathbf{m}_k$ is quicker than matrix assembly plus multiplication with magnetisation vector.

In [24]:
mesh.save("mag.vtk", point_data={"m": mag_init_arr})

## Solve the PDE

In [25]:
from scipy.sparse.linalg import spsolve

In [26]:
rhs = B @ mag_init_flat

Remove the boundary nodes from the system of linear equations since we know that $u = 0$ at the boundary.

In [27]:
S_cnd, rhs_cnd, _, sol_indices = skfem.condense(A=S, b=rhs, D=mesh.boundary_nodes())

In [28]:
sol = spsolve(A=S_cnd, b=rhs_cnd)

In [29]:
np.linalg.norm(S_cnd @ sol - rhs_cnd)

np.float64(2.163012062375222e-11)

## Calculate the $\mathbf{H}_\mathrm{dmg}$ from the scalar potential

In [30]:
pot = function_space_scalar.zeros()

In [31]:
pot[sol_indices] = sol

Project the gradient of the scalar potential to the vector function space on which magnetisation is defined.

In [32]:
h_dmg = function_space_vector.project(-skfem.helpers.grad(function_space_scalar.interpolate(pot)))

In [33]:
h_dmg.shape

(145578,)

In [36]:
h_dmg_reshape = np.stack(np.split(h_dmg, h_dmg.shape[0]/3), axis=0)

In [37]:
h_dmg_reshape.shape

(48526, 3)

In [39]:
mesh.save("h_dmg.vtk", point_data={"h_dmg": h_dmg_reshape})